In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#安裝最佳化套件
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64 -O cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-10-0-local-10.0.130-410.48/7fa2af80.pub
!apt-get update
!apt-get install cuda
!pip install -U tf-nightly
!pip install -U tf-nightly-gpu-2.0-preview
!pip install -U tensorflow-gpu 
!pip install -U tensorflow-model-optimization 

--2020-11-19 12:38:38--  https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/10.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb?APtWxNE9CSlzovFIvUBMrdZighNXZr6HYVGRu5Dfjj5D1cltq8_MqdsuzaUoyqmQ_u1il_JTY6V0OvMUXIencK_pPTNTlyDvHM-zmYAhY62-3gXktNtBHXufIHT6IdwxRXEy0Jeh2GDQYn5egeUHkyByvSyIx67mXN-_V4XDQ6xw-tnZBM0gCnRZkAV2lvNn_q9yrzuCSWQtUNb4Ivcey-q0jolMBnDp1wQRcps [following]
--2020-11-19 12:38:39--  https://developer.download.nvidia.com/compute/cuda/10.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb?APtWxNE9CSlzovFIvUBMrdZighNXZr6HYVGRu5Dfjj5D1cltq8_MqdsuzaU

ERROR: Could not find a version that satisfies the requirement tf-nightly-gpu-2.0-preview (from versions: none)
ERROR: No matching distribution found for tf-nightly-gpu-2.0-preview
     |████████████████████████████████| 320.4MB 55kB/s 
     |████████████████████████████████| 20.1MB 173kB/s 
ERROR: tf-nightly 2.5.0.dev20201118 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.4
    Uninstalling numpy-1.19.4:
      Successfully uninstalled numpy-1.19.4


     |████████████████████████████████| 174kB 37kB/s 


In [3]:
# import package
import matplotlib.pyplot as plt
import numpy as np
import tempfile
import os

import tensorflow as tf
from tensorflow import keras
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
tf.__version__

'2.3.1'

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     BatchNormalization, Flatten,
                                     Conv2D, MaxPooling2D)

In [6]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train.shape:', x_train.shape)
print('y_train.shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 3s 0us/step
x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 1)
50000 train samples
10000 test samples


In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # rescaling
x_test /= 255   # rescaling

In [8]:
# 將訓練資料與測試資料的 label，進行 Onehot encoding 轉換
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_test.argmax(1) shape:', y_test.argmax(1).shape)

y_train shape: (50000, 10)
y_test shape: (10000, 10)
y_test.argmax(1) shape: (10000,)


In [9]:
from keras.models import load_model
VGG_model = load_model('/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model_org.h5')
NASNetLarge_model = load_model('/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model_org.h5')
MobileNetV2_model = load_model('/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model_org.h5')

## 使用 quantization aware API 進行模型訓練


In [10]:
import tensorflow_model_optimization as tfmot

VGG_quantize_model = tfmot.quantization.keras.quantize_model
NASNetLarge_quantize_model = tfmot.quantization.keras.quantize_model
MobileNetV2_quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
VGG_q_aware_model = VGG_quantize_model(VGG_model)
NASNetLarge_q_aware_model = NASNetLarge_quantize_model(NASNetLarge_model)
MobileNetV2_q_aware_model = MobileNetV2_quantize_model(MobileNetV2_model)

# `quantize_model` requires a recompile.
VGG_q_aware_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentrop
              metrics=['accuracy'])

NASNetLarge_q_aware_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentrop
              metrics=['accuracy'])

MobileNetV2_q_aware_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentrop
              metrics=['accuracy'])


IndexError: ignored

In [ ]:
VGG_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentrop
              metrics=['accuracy'])

NASNetLarge_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentrop
              metrics=['accuracy'])

MobileNetV2_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), #SparseCategoricalCrossentrop
              metrics=['accuracy'])


### 訓練並評估比較 (quantization aware training vs. baseline )

在quantization aware training epochs過程中加入兩個callbacks功能，來記錄fine-tuning過程的資訊，雖然是pruning的工具，但仍有記錄功能。
1. 更新training step資訊: `tfmot.sparsity.keras.UpdatePruningStep`
2. 記錄training過程的摘要logs: `tfmot.sparsity.keras.PruningSummaries`

In [ ]:
VGG_logdir = '/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_logs-quant-aware'
NASNetLarge_logdir = '/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_logs-quant-aware'
MobileNetV2_logdir = '/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_logs-quant-aware'

print('Summaries VGG_logdir:', VGG_logdir)
print('Summaries NASNetLarge_logdir:', NASNetLarge_logdir)
print('Summaries MobileNetV2_logdir:', MobileNetV2_logdir)

#建立callbacks
VGG_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=VGG_logdir),
]

NASNetLarge_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=NASNetLarge_logdir),
]

MobileNetV2_callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=MobileNetV2_logdir),
]

In [ ]:
train_images_subset = x_train[0:10000] # out of 50000
train_labels_subset = y_train[0:10000]

VGG_q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=200, epochs=10, validation_split=0.1, callbacks=VGG_callbacks)

NASNetLarge_q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=200, epochs=10, validation_split=0.1, callbacks=NASNetLarge_callbacks)

MobileNetV2_q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=200, epochs=10, validation_split=0.1, callbacks=MobileNetV2_callbacks)

原本Float32 model 訓練為baseline, 與 quantization aware training做比較.

In [ ]:
_, VGG_baseline_model_accuracy = VGG_model.evaluate( x_test, y_test, verbose=0)
_, NASNetLarge_baseline_model_accuracy = NASNetLarge_model.evaluate( x_test, y_test, verbose=0)
_, MobileNetV2_baseline_model_accuracy = MobileNetV2_model.evaluate( x_test, y_test, verbose=0)

_, VGG_q_aware_model_accuracy = VGG_q_aware_model.evaluate( x_test, y_test, verbose=0)
_, NASNetLarge_q_aware_model_accuracy = NASNetLarge_q_aware_model.evaluate( x_test, y_test, verbose=0)
_, MobileNetV2_q_aware_model_accuracy = MobileNetV2_q_aware_model.evaluate( x_test, y_test, verbose=0)

print('VGG_Baseline test accuracy:', VGG_baseline_model_accuracy)
print('VGG_Quant-aware test accuracy:', VGG_q_aware_model_accuracy)

print('NASNetLarge_Baseline test accuracy:', NASNetLarge_baseline_model_accuracy)
print('NASNetLarge_Quant-aware test accuracy:', NASNetLarge_q_aware_model_accuracy)

print('MobileNetV2_Baseline test accuracy:', MobileNetV2_baseline_model_accuracy)
print('MobileNetV2_Quant-aware test accuracy:', MobileNetV2_q_aware_model_accuracy)

## 使用TFLite Converter建立量化模型(quantized model)

真正的將模型轉成 int8 weights 及 uint8 activations

In [ ]:
VGG_converter = tf.lite.TFLiteConverter.from_keras_model(VGG_q_aware_model)
NASNetLarge_converter = tf.lite.TFLiteConverter.from_keras_model(NASNetLarge_q_aware_model)
MobileNetV2_converter = tf.lite.TFLiteConverter.from_keras_model(MobileNetV2_q_aware_model)

VGG_converter.optimizations = [tf.lite.Optimize.DEFAULT]
NASNetLarge_converter.optimizations = [tf.lite.Optimize.DEFAULT]
MobileNetV2_converter.optimizations = [tf.lite.Optimize.DEFAULT]

VGG_tflite_quant_model = VGG_converter.convert()
NASNetLarge_tflite_quant_model = NASNetLarge_converter.convert()
MobileNetV2_tflite_quant_model = MobileNetV2_converter.convert()

## See persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TF Lite model on the test dataset.

In [ ]:
# source by: Copyright 2019 The TensorFlow Authors
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter, test_images, test_labels):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # note: the "digit" is image class id
    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for test_image in test_images:
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    # Compare prediction results with ground truth labels to calculate accuracy.
    accurate_count = 0
    for index in range(len(prediction_digits)):
        if prediction_digits[index].any() == test_labels[index].any():
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)

    return accuracy 

評估比較量化模型在測試集上面的準確率

In [ ]:
VGG_interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_model_quant.tflite')
NASNetLarge_interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_model_quant.tflite')
MobileNetV2_interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_model_quant.tflite')

VGG_interpreter.allocate_tensors()
NASNetLarge_interpreter.allocate_tensors()
MobileNetV2_interpreter.allocate_tensors()

VGG_test_accuracy = evaluate_model(VGG_interpreter, x_test, y_test)
NASNetLarge_test_accuracy = evaluate_model(NASNetLarge_interpreter, x_test, y_test)
MobileNetV2_test_accuracy = evaluate_model(MobileNetV2_interpreter, x_test, y_test)

print('VGG_Baseline test accuracy:', VGG_baseline_model_accuracy)
print('VGG_Quant-aware TF test accuracy:', VGG_q_aware_model_accuracy)
print('VGG_Quant-converter TFLite test_accuracy:', VGG_test_accuracy)

print('NASNetLarge_Baseline test accuracy:', NASNetLarge_baseline_model_accuracy)
print('NASNetLarge_Quant-aware TF test accuracy:', NASNetLarge_q_aware_model_accuracy)
print('NASNetLarge_Quant-converter TFLite test_accuracy:', NASNetLarge_test_accuracy)

print('MobileNetV2_Baseline test accuracy:', MobileNetV2_baseline_model_accuracy)
print('MobileNetV2_Quant-aware TF test accuracy:', MobileNetV2_q_aware_model_accuracy)
print('MobileNetV2_Quant-converter TFLite test_accuracy:', MobileNetV2_test_accuracy)

## 得到一個比原本量化前模型權重檔案小 4倍的量化後模型權重檔案

In [ ]:
# Create float TFLite model.
VGG_float_converter = tf.lite.TFLiteConverter.from_keras_model(VGG_model)
VGG_float_tflite_model = VGG_float_converter.convert()

NASNetLarge_float_converter = tf.lite.TFLiteConverter.from_keras_model(NASNetLarge_model)
NASNetLarge_float_tflite_model = NASNetLarge_float_converter.convert()

MobileNetV2_float_converter = tf.lite.TFLiteConverter.from_keras_model(MobileNetV2_model)
MobileNetV2_float_tflite_model = MobileNetV2_float_converter.convert()

# Measure sizes of models.
VGG_float_file ='/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_float_cifar10_model.tflite'
VGG_quant_file ='/content/drive/MyDrive/Colab Notebooks/edge_AI/VGG/VGG_cifar10_model_quant_aware.tflite'

NASNetLarge_float_file ='/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_float_cifar10_model.tflite'
NASNetLarge_quant_file ='/content/drive/MyDrive/Colab Notebooks/edge_AI/NASNetLarge2/NASNetLarge2_cifar10_model_quant_aware.tflite'

MobileNetV2_float_file ='/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_float_cifar10_model.tflite'
MobileNetV2_quant_file ='/content/drive/MyDrive/Colab Notebooks/edge_AI/MobileNetV2/MobileNetV2_cifar10_model_quant_aware.tflite'

with open(VGG_float_file, 'wb') as f:
    f.write(VGG_float_tflite_model)

with open(VGG_quant_file, 'wb') as f:
    f.write(VGG_tflite_quant_model)


with open(NASNetLarge_float_file, 'wb') as f:
    f.write(NASNetLarge_float_tflite_model)

with open(NASNetLarge_quant_file, 'wb') as f:
    f.write(NASNetLarge_tflite_quant_model)


with open(MobileNetV2_float_file, 'wb') as f:
    f.write(MobileNetV2_float_tflite_model)

with open(MobileNetV2_quant_file, 'wb') as f:
    f.write(MobileNetV2_tflite_quant_model)


print("VGG_Float model in Mb:", os.path.getsize(VGG_float_file) / float(2**20))
print("VGG_Quantized model in Mb:", os.path.getsize(VGG_quant_file) / float(2**20))

print("VGG_Float model in Mb:", os.path.getsize(NASNetLarge_float_file) / float(2**20))
print("VGG_Quantized model in Mb:", os.path.getsize(NASNetLarge_quant_file) / float(2**20))

print("VGG_Float model in Mb:", os.path.getsize(MobileNetV2_float_file) / float(2**20))
print("VGG_Quantized model in Mb:", os.path.getsize(MobileNetV2_quant_file) / float(2**20))


In [ ]:
print("VGG_Float model in Mb:", os.path.getsize(VGG_float_file) / float(2**20))
print("VGG_Quantized model in Mb:", os.path.getsize(VGG_quant_file) / float(2**20))

print("NASNetLarge_Float model in Mb:", os.path.getsize(NASNetLarge_float_file) / float(2**20))
print("NASNetLarge_Quantized model in Mb:", os.path.getsize(NASNetLarge_quant_file) / float(2**20))

print("MobileNetV2_Float model in Mb:", os.path.getsize(MobileNetV2_float_file) / float(2**20))
print("MobileNetV2_Quantized model in Mb:", os.path.getsize(MobileNetV2_quant_file) / float(2**20))
